In [70]:
import pandas as pd



# Example usage
economic_data_path = '/workspaces/Electoral-Economic-Analysis/electoral_analysis/processed_bea_economic_growth.csv'
electoral_data_path = '/workspaces/Electoral-Economic-Analysis/electoral_analysis/processed_electoral_data.csv'


In [71]:
def filtering_economic_indicator(data, indicator, year):
    filtered_data = data[(data['Indicator'] == indicator) & (data['Cycle Duration'] == year)]
    return filtered_data

def filtering_electoral_data(data, year):
    filtered_data = data[data['Year'] == year]
    return filtered_data

# realGDP_data_2009_2013 = filtering_economic_indicator(economic_data, 'Real GDP', '2009-2013')
# election_2008_data =  filtering_electoral_data(electoral_data, 2008)

# print(realGDP_data_2009_2013.head())
# print(election_2008_data.head())

In [72]:
# def normalize_dataset(economic_data, electoral_data, indicator, cycle_duration, election_year):
#     # Normalize the column names for merging
#     economic_data = economic_data.rename(columns={"State": "state", "Cycle Duration": "cycle_duration"})
#     electoral_data = electoral_data.rename(columns={"State": "state", "Year": "year"})

#     # Ensure 'state' columns are in the same format (e.g., uppercase)
#     economic_data['state'] = economic_data['state'].str.upper()
#     electoral_data['state'] = electoral_data['state'].str.upper()

#     # Ensure 'year' columns are integers
#     electoral_data['year'] = electoral_data['year'].astype(int)

#     # Filter economic data for the specified indicator and cycle duration
#     economic_df = economic_data.loc[(economic_data['Indicator'] == indicator) & (economic_data['cycle_duration'] == cycle_duration)].copy()

#     # Adjust the end year to match the election year
#     economic_df.loc[:, 'End Year'] = economic_df['cycle_duration'].str.split('-').str[1].astype(int) - 1

#     # Filter the economic data to match the election year
#     economic_df = economic_df.loc[economic_df['End Year'] == election_year]

#     # Filter electoral data for the specified election year
#     election_df = electoral_data.loc[electoral_data['year'] == election_year]

#     return economic_df, election_df


def normalize_data(economic_data, electoral_data):
    # Normalize the column names for merging
    economic_data = economic_data.rename(columns={"State": "state", "Cycle Duration": "cycle_duration"})
    electoral_data = electoral_data.rename(columns={"State": "state", "Year": "year"})

    # Ensure 'state' columns are in the same format (e.g., uppercase)
    economic_data['state'] = economic_data['state'].str.upper()
    electoral_data['state'] = electoral_data['state'].str.upper()

    # Ensure 'year' columns are integers
    electoral_data['year'] = electoral_data['year'].astype(int)

    # Add 'End Year' column to economic data
    economic_data['End Year'] = economic_data['cycle_duration'].str.split('-').str[1].astype(int) - 1

    return economic_data, electoral_data



AttributeError: 'str' object has no attribute 'rename'

In [73]:

def merge_economic_electoral_data(economic_data, electoral_data, indicator, cycle_duration, election_year):
    # Filter economic data for the specified indicator and cycle duration
    economic_df = economic_data.loc[(economic_data['Indicator'] == indicator) & (economic_data['cycle_duration'] == cycle_duration)].copy()

    # Filter the economic data to match the election year
    economic_df = economic_df.loc[economic_df['End Year'] == election_year]

    # Filter electoral data for the specified election year
    election_df = electoral_data.loc[electoral_data['year'] == election_year]

    # Merge the datasets
    merged_df = pd.merge(economic_df, election_df, left_on=['state', 'End Year'], right_on=['state', 'year'], how='inner')

    # Drop unnecessary columns
    merged_df = merged_df.drop(columns=["End Year"])

    return merged_df

# Example usage
# merged_data = load_and_merge_economic_electoral_data(economic_data_path, electoral_data_path, 'Real GDP', '2009-2013', 2012)
# print(merged_data.head())

In [75]:
def process_all_combinations(economic_data_path, electoral_data_path, indicators, cycle_durations, election_years):
    # Load the datasets
    economic_data = pd.read_csv(economic_data_path)
    electoral_data = pd.read_csv(electoral_data_path)

    # Normalize the datasets
    economic_data, electoral_data = normalize_data(economic_data, electoral_data)

    # Define the correct pairs of cycle durations and election years
    correct_pairs = {
        '2009-2013': 2012,
        '2013-2017': 2016,
        '2017-2021': 2020
    }

    for indicator in indicators:
        for cycle_duration, election_year in correct_pairs.items():
            try:
                merged_data = merge_economic_electoral_data(economic_data, electoral_data, indicator, cycle_duration, election_year)
                output_file = f'/workspaces/Electoral-Economic-Analysis/electoral_analysis/merged_{indicator.replace(" ", "_").lower()}_{cycle_duration}_election_{election_year}.csv'
                merged_data.to_csv(output_file, index=False)
                print(f"Successfully merged {indicator} for {cycle_duration} with election year {election_year}")
            except Exception as e:
                print(f"Error processing {indicator} for {cycle_duration} and {election_year}: {e}")

indicators = ['Real GDP', 'Real Per Capita Personal Income', 'Real Personal Income', 'Real Per Capita PCE', 'Real PCE', 'Total Employment']
cycle_durations = ['2009-2013', '2013-2017', '2017-2021']
election_years = [2012, 2016, 2020]
process_all_combinations(economic_data_path, electoral_data_path, indicators, cycle_durations, election_years)
# Example usage

Successfully merged Real GDP for 2009-2013 with election year 2012
Successfully merged Real GDP for 2013-2017 with election year 2016
Successfully merged Real GDP for 2017-2021 with election year 2020
Successfully merged Real Per Capita Personal Income for 2009-2013 with election year 2012
Successfully merged Real Per Capita Personal Income for 2013-2017 with election year 2016
Successfully merged Real Per Capita Personal Income for 2017-2021 with election year 2020
Successfully merged Real Personal Income for 2009-2013 with election year 2012
Successfully merged Real Personal Income for 2013-2017 with election year 2016
Successfully merged Real Personal Income for 2017-2021 with election year 2020
Successfully merged Real Per Capita PCE for 2009-2013 with election year 2012
Successfully merged Real Per Capita PCE for 2013-2017 with election year 2016
Successfully merged Real Per Capita PCE for 2017-2021 with election year 2020
Successfully merged Real PCE for 2009-2013 with election y

Combined CSV saved to /workspaces/Electoral-Economic-Analysis/electoral_analysis/merged_economic_electoral_result.csv
